In [6]:
from PIL import Image
import numpy as np
from collections import namedtuple
config = namedtuple('config', ['height', 'width'])
config.height = 256
config.width = 256
config.batch_size=8
config.num_epochs=1
val_dir = 'test'
train_dir = 'train'
img_dir = 'images'
from keras import backend as K
import tensorflow as tf

In [7]:

img = Image.open('/home/ubuntu/colorizer-applied-dl/test/amazing-beautiful-beauty-blue.jpg').resize((config.width, config.height))
color_image = np.expand_dims(np.array(img), axis=0)
bw_image = np.expand_dims(np.array(img.convert('L')), axis=0)

In [11]:
def custom_loss_rebalancing_segments(y_true, y_pred):
    y_pred_clipped = K.clip(y_pred, K.epsilon(), None)

    img_height = y_true.shape[1]
    img_width = y_true.shape[2]

    loss = -K.sum(y_true * K.log(y_pred_clipped), axis = 3)

    bin_indices = K.argmax(y_true, axis = 3)
    weights = K.gather(reference = AB_BIN_WEIGHTS, indices = bin_indices)

    loss = RZHANG_LOSS_WEIGHT * K.sum(loss * weights)

    loss += K.sum((y_pred[:, :-1, :, :] - y_pred[:, 1:, :, :]) ** 2)
    loss += K.sum((y_pred[:, :, :-1, :] - y_pred[:, :, 1:, :]) ** 2)

    return loss


In [8]:
y_pred = tf.convert_to_tensor(color_image, np.float32)

In [12]:
y_pred_clipped = K.clip(y_pred, K.epsilon(), None)

In [13]:
y_pred_clipped

<tf.Tensor 'clip_by_value_1:0' shape=(1, 256, 256, 3) dtype=float32>

In [19]:
y_true = tf.convert_to_tensor(np.array(Image.open('/home/ubuntu/colorizer-applied-dl/test/art-creative-flowers-244497.jpg').resize((config.width, config.height))), np.float32)

In [20]:
loss = -K.sum(y_true * K.log(y_pred_clipped), axis = 3)

In [23]:
y_true.shape

TensorShape([Dimension(256), Dimension(256), Dimension(3)])